In [123]:
import pandas as pd
import numpy as np
import re
import itertools

# What do I do?

I import the data, I group it by District and Verkiezingsdatum (so as to have unique groups), and then I calculate positions. 

Then I calculate the winning and losing candidates by taking "if <position> > <Aantal zetels>" then Loser, otherwise winner. 
    
Then, I calculate the margins for the losers to the lowest winner. 
    
Finally, I make a new dataset containing only the closely elected politicians and the names of the winners of the district.

In [124]:
data = pd.read_csv("../Data/elections/election_results_details.csv", encoding='latin-1')
data = data.drop('Unnamed: 0', axis=1)

## First, make a counter

In [125]:
def counter(group):
    temp = pd.Series(range(0,len(group)), dtype='int')
    temp = temp + 1
    return temp

In [126]:
data["counts"] = data.groupby(['District', 'Verkiezingdatum'], sort = False)['Naam'].transform(counter)

## Then, make win indicators

In [127]:
data['Aantal zetels'] = pd.to_numeric(data['Aantal zetels'], errors='coerce')

data["won"] = np.where(data["counts"] <= data['Aantal zetels'], 1,0)

## Now, calculate the margin to the lowest winner

In [128]:
winners = data.groupby(['District', 'Verkiezingdatum'], sort = False, as_index = False).apply(lambda x: x[x['won'] == 1])
winners['Aantal stemmen'] = pd.to_numeric(winners['Aantal stemmen'])

In [129]:
minwinners = winners.loc[winners.groupby(['District', 'Verkiezingdatum'], sort = False, as_index = False)['Aantal stemmen'].idxmin()]
minwinners = minwinners.reset_index().iloc[:,2:]

# Now

Make a per unique election min_procentueel and min_aantal_stemmen

Then merge them with the large dataframe with all candidates

And calculate the margn

If negative, then they lost. We then proceed to check a couple of things

In [130]:
#Make a per unique election min_procentueel and min_aantal_stemmen
minwinners = minwinners.iloc[:,[4,5,2,3]]

minwinners = minwinners.rename({'Aantal stemmen':'min_stem_win', 
                  'Procentueel':'min_pct_win'}, axis=1)

In [131]:
#Then merge them with the large dataframe with all candidates
allmgn = data.merge(minwinners, how='left', left_on=['District','Verkiezingdatum'], right_on=['District','Verkiezingdatum'])

In [132]:
#Calculate the margin
allmgn['Aantal stemmen'] = pd.to_numeric(allmgn['Aantal stemmen'], errors='coerce')

def p2f(x):
    var = []
    for i in x:
        var.append(pd.to_numeric(i.strip('%'), errors='coerce')/100)
    return var

allmgn['Procentueel'] = p2f(allmgn['Procentueel'])

In [133]:
def p2f_nan(x):
    var = []
    for i in x:
        if type(i) == str:
            var.append(pd.to_numeric(i.strip('%'), errors='coerce')/100)
        else:
            var.append(i)
    return var

allmgn['min_pct_win'] = p2f_nan(allmgn['min_pct_win'])

In [134]:
allmgn['margin'] = allmgn['Procentueel'] - allmgn['min_pct_win']

In [135]:
# If negative, they lost:

def close_cand(x, pct):
    var = []
    for i in x:
        var.append(pd.to_numeric(np.where(i < 0 and i > -pct, 1, 0)))
    return pd.Series(var)
        

allmgn['close_cand_5pct'] = close_cand(allmgn['margin'], 0.05)
allmgn['close_cand_10pct'] = close_cand(allmgn['margin'], 0.10)
allmgn['close_cand_20pct'] = close_cand(allmgn['margin'], 0.20)

In [170]:
allmgn['Kiesdrempel'] = pd.to_numeric(allmgn['Kiesdrempel'], errors='coerce')
allmgn['Aantal stemmen geldig'] = pd.to_numeric(allmgn['Aantal stemmen geldig'], errors='coerce')

#allmgn[allmgn['Aantal stemmen geldig'] > allmgn['Kiesdrempel']]
#Everything is larger than kiesdrempel

## Finally..

Take the data frame, take only the close losers

- For each close loser, look up the (just) winner in that election (only 1 person)
- Match each loser to that winner (multiple matches per district possible if e.g. 2 close losing candidates)
    - Merge strategy: take all the eligible just losers
    - Merge them to a filtered dataset finished with the just winners on the basis of district, date (left join with left = just losers)



In [171]:
# All just Losers
justlosers = allmgn[allmgn['close_cand_20pct'] == 1]


# Filtered dataset with just winners
justwinners = allmgn[allmgn['counts'] == allmgn['Aantal zetels']]

# Filtered dataset with all winners
allwinners = allmgn[allmgn['counts'] <= allmgn['Aantal zetels']]

In [172]:
#change the varnames for the justwinners to avoid duplicate issues
justwinners = justwinners.reset_index().iloc[:,1:7].rename({
    'Naam':'Naam_win',
    'Aanbevolen door':'Aanbevolen door_win',
    'Aantal stemmen':'Aantal stemmen_win',
    'Procentueel':'Procentueel_win'}, axis=1)

In [173]:
# Merge justwinners with just losers
closepairs = justlosers.merge(justwinners, how='left', on=['District','Verkiezingdatum'])

# Retrieve some additional information

- Are they on parlement.com?
- Sterfplaats
- Sterfteleeftijd (geboortejaar)
- Distance to Den Haag via Google Maps API

In [182]:
#J'ai utilise ce fichier auparavant pour générer data_entry.csv sur Google Drive
#Maintenant j'en fais un fichier plus précis, quite à omettre quelques matches potentiels 

#closepairs.to_csv("../Administration/close_pairs.csv")

politici = allwinners['Naam'].drop_duplicates()
#politici = politici.drop_duplicates()

closepairs['alrdy_in_pols'] = np.where(closepairs['Naam'].isin(politici),'ja','nee')

closepairs2 = closepairs[closepairs['alrdy_in_pols'] == 'nee']

In [190]:
closepairs2.to_csv("../Administration/close_pairs2.csv")